# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686606


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:06<01:40,  3.47s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<01:06,  2.45s/it]

Rendering models:  19%|█▉        | 6/31 [00:07<00:43,  1.76s/it]

Rendering models:  26%|██▌       | 8/31 [00:07<00:28,  1.26s/it]

...

Rendering models:  81%|████████  | 25/31 [00:09<00:00,  6.77it/s]

Rendering models:  87%|████████▋ | 27/31 [00:09<00:00,  7.75it/s]

Rendering models:  94%|█████████▎| 29/31 [00:09<00:00,  7.94it/s]

Rendering models: 100%|██████████| 31/31 [00:10<00:00,  8.31it/s]

not-logged-in-552245d05ec0c4718543    0.001808
not-logged-in-8e0ce538cfce67a074d4    0.000562
not-logged-in-f6fac0763296d5759c01    0.003257
Almudys                               0.003256
not-logged-in-fceaf6f9645a2f020c34    0.032905
Dynamatt                              0.000548
SavinSpaceEngineer                    0.000680
kjkavene                              0.000490
not-logged-in-c3e2bb63f9ec3bdeea91    0.000644
CThomas                               0.001443
ferlin1998                            0.005230
Jessica_Korkmaz                       0.004358
Samuel_Liuwe                          0.000867
Maverna                               0.045776
withadnotat                           0.000821
Saharisunshine                        0.001322
not-logged-in-214814e52004b42491a4    0.022101
BraxtonKinney                         0.000775
Lavadude                              0.050948
not-logged-in-4ea23f5fffc8b98635af    0.001024
clars915                              0.000954
atayl233     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())